In [2]:
import numpy as np
import pandas as pd

In [ ]:
df_regions = pd.read_csv('noc_regions.csv')
df_regions.head()

In [ ]:
df = pd.read_csv('athlete_events.csv')
df.head()

**As i am only doing analysis of the Summer Olympics so I'll filter out the entries of Winter olypmics**

In [ ]:
df = df[df['Season'] == 'Summer']

In [ ]:
df.columns

Column Name | Description
------------|--------------
ID | Unique number for each athlete
Name | Name of the athlete
Sex | Gender M or F
Age | Age of the athlete
Height | Height of the athlete
Weight | Weight of the athlete
Team | Team name
NOC | National Olympic Committee 3-letter code
Games | Year and season
Year | Year of the Olympics games
Season | Season of the Olympics games
City | Host city
Sport | Sport Name
Event | Categories of Events
Medal | Gold, Silver, Bronze, or NA
Region | Name of the country


In [ ]:
# Merging the two datasets to get Name of the countries
df = df.merge(df_regions,on='NOC',how='left')
df.head()

In [ ]:
# Checking for Null Values
null_values = (df.isnull().sum()/df.shape[0])*100
print('\tPercentage of Null Values in Each Column \n\n',null_values)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df = pd.concat([df,pd.get_dummies(df['Medal'])],axis=1)
df.head()

Now we tally the number of medals won by each country in the Summer Olympics.

In [ ]:
medals = df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])
medals.shape

In [ ]:
medal_tally = medals.groupby('NOC').sum()[['Gold','Silver','Bronze']].sort_values('Gold',ascending=False).reset_index()
medal_tally['Total'] = medal_tally['Gold'] + medal_tally['Silver'] + medal_tally['Bronze']


In [ ]:
medal_tally[['Total','Gold','Silver','Bronze']]=medal_tally[['Total','Gold','Silver','Bronze']].astype('int')

In [ ]:
years = df['Year'].unique().tolist()

In [ ]:
years.sort()

In [ ]:
years.insert(0,'OverAll')

In [ ]:
country=np.unique(df['region'].dropna().values).tolist()

In [ ]:
country.sort()

In [ ]:
country.insert(0,'OverAll')

In [ ]:
def fetch_medal_tally(df,year,country):
    
    flag=0
    
    medals = df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport', 'Event','Medal'])
    
    if year == 'OverAll' and country == 'OverAll':
        temp_df = medals
    elif (year !='OverAll') and (country == 'OverAll'):
        temp_df = medals[medals['Year']==int(year)]
    elif (year == 'OverAll') and (country != 'OverAll'):
        flag=1
        temp_df = medals[medals['region']==country]
    elif (year != 'OverAll') and (country != 'OverAll'):
        temp_df = medals[(medals['Year']==int(year)) & (medals['region']==country)]
    
    if flag == 1:
        x = temp_df.groupby('Year').sum()[['Gold','Silver','Bronze']].sort_values('Year').reset_index()    
    else:
        x = temp_df.groupby('region').sum()[['Gold','Silver','Bronze']].sort_values('Gold',ascending=False).reset_index()
    x['Total'] = x['Gold'] + x['Silver'] + x['Bronze']
    
    x[['Total','Gold','Silver','Bronze']]=x[['Total','Gold','Silver','Bronze']].astype('int')
    
    return x

### OverAll Analysis

 * No.of Editions
 * No.of Cities
 * No.of Events/Sports
 * No.of Athletes
 * No.of Participating Countries
 * No.of Medals
 * Events Played Over the Years
 * Athletes Participated Over the Years
 * Participating countries Over the Years
 * No.of Events/Sports Over the Years
 * Most Successful Athletes


In [ ]:
no_of_editions = df['Year'].unique().shape[0]-1 #Subtracting 1 because Olympics 1906 was Nullified
no_of_cities = df['City'].unique().shape[0] #No. of Cities
no_of_sports = df['Sport'].unique().shape[0] # No. of Sports Played in Olympics
no_of_events = df['Event'].unique().shape[0] # No. of Events 
No_of_athletes = df['Name'].unique().shape[0] # No. of athletes 
no_of_countries = df['region'].unique().shape[0] # No. of Countries participated

In [ ]:
# Plotting a Line graph of Participating nations over the year

import plotly.express as px

nations_over_time = df.drop_duplicates(['Year','region'])['Year'].value_counts().reset_index().sort_values('index')
nations_over_time.rename(columns={'index':'Year','Year':'No_of_countries'},inplace=True)

fig = px.line(nations_over_time,x='Year',y='count')
fig.show()

In [ ]:
# Plotting Number of events over time

events_over_years = df.drop_duplicates(['Year','Event'])['Year'].value_counts().reset_index().sort_values('index')
events_over_years.rename(columns={'index':'Year','Year':'Events'},inplace=True)

fig = px.line(events_over_years,x='Year',y='Events')
fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

x = df.drop_duplicates(['Year','Sport','Event'])

plt.figure(figsize=(25,25))
sns.heatmap(x.pivot_table(index = 'Sport' , columns = 'Year' , values = 'Event', aggfunc='count').fillna(0).astype(int),annot=True)
plt.show()

In [ ]:
def athlete_success(df,sport):
    temp_df = df.dropna(subset=['Medal'])
    if sport != 'OverAll':
        temp_df = temp_df[temp_df['Sport']==sport]
    x = temp_df['Name'].value_counts().reset_index().head(10).merge(df,left_on='index',right_on='Name',how='left')[['index','Name_x','Sport','region']].drop_duplicates()
    x.rename(columns={'index':'Name','Name_x':'Medals'},inplace=True)
    return x

In [ ]:
athlete_success(df,'OverAll')